In [ ]:
import asyncio
from jupyasyncclient import JupyAsyncKernelClient

## Quick start

In [ ]:
host,port = '127.0.0.1',9999
url = f"http://{host}:{port}"
token = 'x'

In [ ]:
km = JupyAsyncKernelManager(url, token=token)
await km.start_kernel()
kc = km.client()
kc.start_channels()
await kc.wait_for_ready(timeout=30)

Or:
```py
km, kc = await start_new_server_kernel(url, token=token)
```

In [ ]:
rep = await kc.execute_interactive("print('hello')", timeout=30)
print(rep["content"]["status"])

In [ ]:
await kc.aclose()
await km.shutdown_kernel(now=True)
await km.aclose()